In [1]:


# get all data in a single dataframe

#     a) collects all the available monthly CSV datasets to make longer monthly analysis possible

import requests
from bs4 import BeautifulSoup

page = "https://www.censtatd.gov.hk/hkstat/sub/sp140.jsp?productCode=B1010002"
req = requests.get(page)
soup = BeautifulSoup(req.content)
df = soup.find("table",id="df")
links = df.find_all("a")
csvs = set()
for link in links: # start from the second link to remove the duplicate
    if link.text == 'CSV Datasets':
        csvs.update([link.get('href')])
for link in links: # start from the second link to remove the duplicate
    if link.text == 'CSV Datasets':
        latest = link.get('href').replace("/fd.jsp?file=","").replace(".zip&product_id=B1010002&lang=1","")
        print(latest)
        break
csvs
    


B10100022020MM12B


{'/fd.jsp?file=B10100022019MM11B.zip&product_id=B1010002&lang=1',
 '/fd.jsp?file=B10100022019MM12B.zip&product_id=B1010002&lang=1',
 '/fd.jsp?file=B10100022020MM01B.zip&product_id=B1010002&lang=1',
 '/fd.jsp?file=B10100022020MM02B.zip&product_id=B1010002&lang=1',
 '/fd.jsp?file=B10100022020MM03B.zip&product_id=B1010002&lang=1',
 '/fd.jsp?file=B10100022020MM04B.zip&product_id=B1010002&lang=1',
 '/fd.jsp?file=B10100022020MM05B.zip&product_id=B1010002&lang=1',
 '/fd.jsp?file=B10100022020MM06B.zip&product_id=B1010002&lang=1',
 '/fd.jsp?file=B10100022020MM07B.zip&product_id=B1010002&lang=1',
 '/fd.jsp?file=B10100022020MM08B.zip&product_id=B1010002&lang=1',
 '/fd.jsp?file=B10100022020MM09B.zip&product_id=B1010002&lang=1',
 '/fd.jsp?file=B10100022020MM10B.zip&product_id=B1010002&lang=1',
 '/fd.jsp?file=B10100022020MM11B.zip&product_id=B1010002&lang=1',
 '/fd.jsp?file=B10100022020MM12B.zip&product_id=B1010002&lang=1'}

In [2]:
import datetime
import zipfile

# create folders for today's downloads
today = datetime.datetime.now().strftime("%Y-%m-%d") 
!mkdir download
!mkdir data
!mkdir download/$today
!mkdir data/$today

def download_url(url, save_path, chunk_size=128):
    r = requests.get(url, stream=True)
    with open(save_path, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=chunk_size):
            fd.write(chunk)

for csv in csvs:
    name = csv.replace("/fd.jsp?file=","").replace(".zip&product_id=B1010002&lang=1","")
    download_url("https://www.censtatd.gov.hk"+csv,"download/"+today+"/"+name+".zip")
    
    with zipfile.ZipFile("download/"+today+"/"+name+".zip", 'r') as zip_ref:
        zip_ref.extractall("data/"+today+"/"+name)

mkdir: download: File exists
mkdir: data: File exists


In [3]:
files = !cd data/$today/B10100022019MM11B;ls
dfiles = []
for file in files:
    if "master" in file:
        dfiles.append(file)
dfiles

['B10100022019MM11B01-Data master.CSV',
 'B10100022019MM11B02-Data master.CSV',
 'B10100022019MM11B03-Data master.CSV',
 'B10100022019MM11B04-Data master.CSV',
 'B10100022019MM11B05-Data master.CSV',
 'B10100022019MM11B06-Data master.CSV',
 'B10100022019MM11B07-Data master.CSV',
 'B10100022019MM11B08-Data master.CSV',
 'B10100022019MM11B09-Data master.CSV',
 'B10100022019MM11B10-Data master.CSV',
 'B10100022019MM11B11-Data master.CSV']

In [19]:
import pandas as pd
bigdf = pd.DataFrame()
bigdesc = pd.DataFrame()
for file in dfiles:
    print(file)
    df = pd.read_csv("data/%s/B10100022019MM11B/%s"%(today,file))
    df['file'] = file
    bigdf = bigdf.append(df)
    bigdf['period'] = bigdf['period'].astype(str)
    desc = pd.read_csv("data/%s/B10100022019MM11B/%s"%(today,file.replace("master","description")))
    desc['file'] = file
    bigdesc = bigdesc.append(desc)

B10100022019MM11B01-Data master.CSV
B10100022019MM11B02-Data master.CSV
B10100022019MM11B03-Data master.CSV
B10100022019MM11B04-Data master.CSV
B10100022019MM11B05-Data master.CSV
B10100022019MM11B06-Data master.CSV
B10100022019MM11B07-Data master.CSV
B10100022019MM11B08-Data master.CSV
B10100022019MM11B09-Data master.CSV
B10100022019MM11B10-Data master.CSV
B10100022019MM11B11-Data master.CSV


In [20]:
bigdesc

,product,issue,table,category,code,desc_c,desc_e,file
0,HKMDS,201911,NaN,product,HKMDS,香港統計月刊,Hong Kong Monthly Digest of Statistics,B10100022019MM11B01-Data master.CSV
1,HKMDS,201911,NaN,issue,201911,期號：2019年11月,Issue : November 2019,B10100022019MM11B01-Data master.CSV
2,HKMDS,201911,T0101,table,T0101,表1.1：年中及年底人口,Table 1.1 : Mid-year and year-end population,B10100022019MM11B01-Data master.CSV
3,HKMDS,201911,T0101,stat,M010101,人口,Population,B10100022019MM11B01-Data master.CSV
4,HKMDS,201911,T0101,class1,sex,性別,Sex,B10100022019MM11B01-Data master.CSV
...,...,...,...,...,...,...,...,...
384,HKMDS,201911,T1113,stat,M111303,廣東省供水量 [百萬立方米],Imports from Guangdong [Million cubic metres],B10100022019MM11B11-Data master.CSV
385,HKMDS,201911,T1113,stat,M111304,耗水量 - 總計 [百萬立方米],Consumption - Total [Million cubic metres],B10100022019MM11B11-Data master.CSV
386,HKMDS,201911,T1113,stat,M111305,每日平均耗水量 [百萬立方米],Consumption - Daily mean [Million cubic metres],B10100022019MM11B11-Data master.CSV
387,HKMDS,201911,T1113,note_figure,(+),工程維修保養期間，廣東省暫停供水。,The supply from Guangdong was shutdown for mai...,B10100022019MM11B11-Data master.CSV


In [51]:
month = bigdf[bigdf['period'].isin(['01','02','03','04','05','06','07','08','09','10','11','12'])]
check = pd.DataFrame(month.groupby(['stat','class1_code','class2_code','class3_code','class4_code','file'],dropna=False).size())
checked = check[check[0]==15]
checked.columns = ['count']
checked = checked.reset_index()

In [55]:
checked

,stat,class1_code,class2_code,class3_code,class4_code,file,count
0,M010401,sex_01,NaN,NaN,NaN,B10100022019MM11B01-Data master.CSV,15
1,M010401,sex_02,NaN,NaN,NaN,B10100022019MM11B01-Data master.CSV,15
2,M010401,sex_tt,NaN,NaN,NaN,B10100022019MM11B01-Data master.CSV,15
3,M010402,sex_01,NaN,NaN,NaN,B10100022019MM11B01-Data master.CSV,15
4,M010402,sex_02,NaN,NaN,NaN,B10100022019MM11B01-Data master.CSV,15
...,...,...,...,...,...,...,...
1037,M111301,NaN,NaN,NaN,NaN,B10100022019MM11B11-Data master.CSV,15
1038,M111302,NaN,NaN,NaN,NaN,B10100022019MM11B11-Data master.CSV,15
1039,M111303,NaN,NaN,NaN,NaN,B10100022019MM11B11-Data master.CSV,15
1040,M111304,NaN,NaN,NaN,NaN,B10100022019MM11B11-Data master.CSV,15


In [63]:
coded = checked
for row in bigdesc.set_index('code')['desc_e'].iteritems():
    coded = coded.replace(row[0],row[1])
coded

,stat,class1_code,class2_code,class3_code,class4_code,file,count
0,Registered births,Male,NaN,NaN,NaN,B10100022019MM11B01-Data master.CSV,15
1,Registered births,Female,NaN,NaN,NaN,B10100022019MM11B01-Data master.CSV,15
2,Registered births,Total,NaN,NaN,NaN,B10100022019MM11B01-Data master.CSV,15
3,Registered deaths (note 1)(note 2),Male,NaN,NaN,NaN,B10100022019MM11B01-Data master.CSV,15
4,Registered deaths (note 1)(note 2),Female,NaN,NaN,NaN,B10100022019MM11B01-Data master.CSV,15
...,...,...,...,...,...,...,...
1037,Storage at beginning of period [Million cubic ...,NaN,NaN,NaN,NaN,B10100022019MM11B11-Data master.CSV,15
1038,Natural yield in reservoirs [Million cubic met...,NaN,NaN,NaN,NaN,B10100022019MM11B11-Data master.CSV,15
1039,Imports from Guangdong [Million cubic metres],NaN,NaN,NaN,NaN,B10100022019MM11B11-Data master.CSV,15
1040,Consumption - Total [Million cubic metres],NaN,NaN,NaN,NaN,B10100022019MM11B11-Data master.CSV,15


In [64]:
checked

,stat,class1_code,class2_code,class3_code,class4_code,file,count
0,M010401,sex_01,NaN,NaN,NaN,B10100022019MM11B01-Data master.CSV,15
1,M010401,sex_02,NaN,NaN,NaN,B10100022019MM11B01-Data master.CSV,15
2,M010401,sex_tt,NaN,NaN,NaN,B10100022019MM11B01-Data master.CSV,15
3,M010402,sex_01,NaN,NaN,NaN,B10100022019MM11B01-Data master.CSV,15
4,M010402,sex_02,NaN,NaN,NaN,B10100022019MM11B01-Data master.CSV,15
...,...,...,...,...,...,...,...
1037,M111301,NaN,NaN,NaN,NaN,B10100022019MM11B11-Data master.CSV,15
1038,M111302,NaN,NaN,NaN,NaN,B10100022019MM11B11-Data master.CSV,15
1039,M111303,NaN,NaN,NaN,NaN,B10100022019MM11B11-Data master.CSV,15
1040,M111304,NaN,NaN,NaN,NaN,B10100022019MM11B11-Data master.CSV,15
